In [2]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds

In [ ]:
class ActionDataset(Dataset):
    def __init__(self,Data):
        super().__init__()
        self.N_frame=Data['total_frame']
        image_shape=Data[]
        self.kp=Data['kp'][0]/

    def __getitem__(self, index):
        return super().__getitem__(index)

    def __len__(self):
        return     

In [ ]:
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])